## Turtlebot3 robot model and spawn

This tutorial course follows the Udemy course:
https://github.com/noshluk2/7_Autonomous_driving_navigation_stack


The objectives are:
- Describe the Turtlebot3 wafle robot and spawn in a world
- perform Navigation
- perform SLAM

The Turtlebot3 waffle is very popular and has interesting navigation sensors:
- Raspberry pi camera
- RPLIDAR

<img src="./Images/1_turtlebot3.png">

### Install needed packages

We need to install different ROS packages to perform Navigation and SLAM

In [ ]:
sudo apt-get install ros-melodic-navigation
sudo apt-get install ros-melodic-slam-gmapping
sudo apt-get install ros-melodic-teleop-twist-keyboard
sudo apt-get install ros-melodic-turtlebot3-msgs
sudo apt-get install ros-melodic-turtlebot3

### Create the workspace

We create now a "turtlebot3_pc_ws" workspace to work with two packages:
- turtlebot3_simulations 
- autonomous robots

Let's create first the ws:

In [ ]:
mkdir -p ~/turtlebot3_pc_ws/src
cd turtlebot3_pc_ws
catkin_make

Clone the "turtlebot3_simulations" package from https://github.com/ROBOTIS-GIT/turtlebot3_simulations

This package has to be cloned and compiled every time you download the repository because the git packages are not automatically upload to your github repository. If you want to syncronize and upload the package, you have to delete the ".git" folder in this package.

In [ ]:
cd ~/turtlebot3_pc_ws/src
git clone https://github.com/ROBOTIS-GIT/turtlebot3_simulations.git

Create a new package "autonomous_robots" with rospy, roscpp, std_msgs dependencies

In [ ]:
catkin_create_pkg autonomous_robots std_msgs rospy roscpp
cd ~/turtlebot3_pc_ws
catkin_make

Create in this package folder the following folders:
- launch: for launch files
- map: for the generated maps in SLAM processes
- world: for designed worlds
- scripts: for python files to perform speciffic control objectives

Finally insert in the .bashrc file the following code lines (verify your IP address)

In [ ]:
source /opt/ros/melodic/setup.bash
source ~/turtlebot3_pc_ws/devel/setup.bash
export TURTLEBOT3_MODEL=waffle_pi
export ROS_IP=192.168.18.84
export ROS_MASTER_URI=http://192.168.18.84:11311


There are 2 softwares to simulate the robot behaviour:
- rviz: this software is able to view the robot model and all the topic messages information
- gazebo: this software is a real dynamic simulator and you will be able to interact physically with a simulated world

<img src="./Images/1_sw.png">

### Spawn the turtlebot3 robot

To spawn our turtlebot3 robot in world we will need to read the TF frames of the different links and joints of the robot and publish them in rviz software. This is done in "robot_state_publisher" package.

<img src="./Images/1_spawn.png">

To spawn our turtlebot3 robot in world we will dreate a launch file "l1_robot.launch"

In [ ]:
<launch>
     <!-- gazebo launch with empty world -->
    <include file ="$(find gazebo_ros)/launch/empty_world.launch"/>

    <!-- uploading urdf robot model into parameter server -->
    <param name="robot_description" command="$(find xacro)/xacro --inorder $(find turtlebot3_description)/urdf/turtlebot3_waffle_pi.urdf.xacro"/>  
      
     <!-- from parameter server spawn model into gazebo  -->
    <node pkg="gazebo_ros" type="spawn_model" name="spawn_urdf" args="-urdf -model turtlebot3_waffle_pi -param robot_description">
    </node>
     <!-- publisher transform for all joint and thier inbetween relations -->
    <node pkg="robot_state_publisher" type="robot_state_publisher" name="robot_state_publisher">
    <param name ="publish_frequency" type=" double" value="50.0"/>
    </node>
    <!-- Running the robot moving node -->
    <node pkg="teleop_twist_keyboard" type="teleop_twist_keyboard.py" name="teleop_twist_keyboard">
        
    </node>

</launch>

- execute this launch file to spawn the robot:

roslaunch autonomous_robots l1_robot.launch

- you can insert objects in your world to interact with the robot sensors.

- You can see the topic list

- you can open the rviz and view:
    - the robot because you have started the "robot_state_publisher" node
    - the sensor readings (laserscan and camera)

<img src="./Images/1_gazebo_rviz.png">

To close the gazebo simulation type:

killall gzserver && killall gzclient

## SLAM

Simultaneous Localisation and Mapping techniques are used to autonomous navigate from an initial point to a target point avoiding obstacles. The first step is the mapping generation using speciffic LIDAR sensor.

For this purpose a speciffic turtlebot SLAM node has to be launched to generate the map.

<img src="./Images/1_slam.png">

The turtlebot3 SLAM node is created in the "turtlebot3_slam" package installed in our ROS environment. The launch file "turtlebot3_slam.launch" executes this node to generate the map.

Let's create a new "l2_SLAM.launch" launch file including the execution of this "turtlebot3_slam.launch" file

In [ ]:
<launch>
     <!-- gazebo launch with empty world and loading the new model we have created-->
    <include file ="$(find gazebo_ros)/launch/empty_world.launch">
        <arg name="world_name" value="$(find autonomous_robots)/world/office_tb3.world"/>   
    </include>
    <!-- uploading urdf robot model into parameter server -->
    <param name="robot_description" command="$(find xacro)/xacro --inorder $(find turtlebot3_description)/urdf/turtlebot3_waffle_pi.urdf.xacro"/>  
      
    <!-- Running the robot moving node -->
    <node pkg="teleop_twist_keyboard" type="teleop_twist_keyboard.py" name="teleop_twist_keyboard">
    </node>
    <!-- Slam Node running for creating map -->
    <include file="$(find turtlebot3_slam)/launch/turtlebot3_slam.launch">
        <arg name="slam_methods" value="gmapping"/>
    </include>    
</launch>

Execute the previous launch file:

roslaunch autonomous_robots l2_SLAM.launch